<a href="https://colab.research.google.com/github/Khushil-Modi/CMPE-297/blob/main/Assignment%204/CMPE_297_Assignment_4(c)_Meta_Learning_on_top_of_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Dataset & Install Libraries

In [1]:
# Let inspect the data
import json
import requests
from random import shuffle
reviews = requests.get('https://raw.githubusercontent.com/mailong25/meta-learning-bert/master/dataset.json')
reviews = reviews.json()

reviews[:5]

[{'text': "GOOD LOOKING KICKS IF YOUR KICKIN IT OLD SCHOOL LIKE ME. AND COMFORTABLE. AND RELATIVELY CHEAP. I'LL ALWAYS KEEP A PAIR OF STAN SMITH'S AROUND FOR WEEKENDS",
  'label': 'positive',
  'domain': 'apparel'},
 {'text': 'These sunglasses are all right. They were a little crooked, but still cool..',
  'label': 'positive',
  'domain': 'apparel'},
 {'text': "I don't see the difference between these bodysuits and the more expensive ones. Fits my boy just right",
  'label': 'positive',
  'domain': 'apparel'},
 {'text': 'Very nice basic clothing. I think the size is fine. I really like being able to find these shades of green, though I have decided the lighter shade is really a feminine color. This is the only brand that I can find these muted greens',
  'label': 'positive',
  'domain': 'apparel'},
 {'text': 'I love these socks. They fit great (my 15 month old daughter has thick ankles) and she can zoom around on the kitchen floor and not take a nose dive into things. :',
  'label': 'p

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 63.9 MB/s 
     |████████████████████████████████| 7.6 MB 57.6 MB/s 


In [3]:
from collections import Counter
mention_domain = [r['domain'] for r in reviews]
Counter(mention_domain)

Counter({'apparel': 1717,
         'baby': 1107,
         'beauty': 993,
         'books': 921,
         'camera_&_photo': 1086,
         'cell_phones_&_service': 698,
         'dvd': 893,
         'electronics': 1277,
         'grocery': 1100,
         'health_&_personal_care': 1429,
         'jewelry_&_watches': 1086,
         'kitchen_&_housewares': 1390,
         'magazines': 1133,
         'music': 1007,
         'outdoor_living': 980,
         'software': 1029,
         'sports_&_outdoors': 1336,
         'toys_&_games': 1363,
         'video': 1010,
         'automotive': 100,
         'computer_&_video_games': 100,
         'office_products': 100})

###Import Libraries & Create Tasks

In [4]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import collections
import random
import json, pickle
from torch.utils.data import TensorDataset

LABEL_MAP  = {'positive':0, 'negative':1, 0:'positive', 1:'negative'}

class MetaTask(Dataset):
    
    def __init__(self, examples, num_task, k_support, k_query, tokenizer):
        """
        :param samples: list of samples
        :param num_task: number of training tasks.
        :param k_support: number of support sample per task
        :param k_query: number of query sample per task
        """
        self.examples = examples
        random.shuffle(self.examples)
        
        self.num_task = num_task
        self.k_support = k_support
        self.k_query = k_query
        self.tokenizer = tokenizer
        self.max_seq_length = 128
        self.create_batch(self.num_task)
    
    def create_batch(self, num_task):
        self.supports = []  # support set
        self.queries = []  # query set
        
        for b in range(num_task):  # for each task
            # 1.select domain randomly
            domain = random.choice(self.examples)['domain']
            domainExamples = [e for e in self.examples if e['domain'] == domain]
            
            # 1.select k_support + k_query examples from domain randomly
            selected_examples = random.sample(domainExamples,self.k_support + self.k_query)
            random.shuffle(selected_examples)
            exam_train = selected_examples[:self.k_support]
            exam_test  = selected_examples[self.k_support:]
            
            self.supports.append(exam_train)
            self.queries.append(exam_test)

    def create_feature_set(self,examples):
        all_input_ids      = torch.empty(len(examples), self.max_seq_length, dtype = torch.long)
        all_attention_mask = torch.empty(len(examples), self.max_seq_length, dtype = torch.long)
        all_segment_ids    = torch.empty(len(examples), self.max_seq_length, dtype = torch.long)
        all_label_ids      = torch.empty(len(examples), dtype = torch.long)

        for id_,example in enumerate(examples):
            input_ids = tokenizer.encode(example['text'])
            attention_mask = [1] * len(input_ids)
            segment_ids    = [0] * len(input_ids)

            while len(input_ids) < self.max_seq_length:
                input_ids.append(0)
                attention_mask.append(0)
                segment_ids.append(0)

            label_id = LABEL_MAP[example['label']]
            all_input_ids[id_] = torch.Tensor(input_ids).to(torch.long)
            all_attention_mask[id_] = torch.Tensor(attention_mask).to(torch.long)
            all_segment_ids[id_] = torch.Tensor(segment_ids).to(torch.long)
            all_label_ids[id_] = torch.Tensor([label_id]).to(torch.long)

        tensor_set = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)  
        return tensor_set
    
    def __getitem__(self, index):
        support_set = self.create_feature_set(self.supports[index])
        query_set   = self.create_feature_set(self.queries[index])
        return support_set, query_set

    def __len__(self):
        # as we have built up to batchsz of sets, you can sample some small batch size of sets.
        return self.num_task

###Split meta training and meta testing

In [5]:
low_resource_domains = ["office_products", "automotive", "computer_&_video_games"]
train_examples = [r for r in reviews if r['domain'] not in low_resource_domains]
test_examples = [r for r in reviews if r['domain'] in low_resource_domains]
print(len(train_examples), len(test_examples))

21555 300


In [6]:
import torch
from transformers import BertModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
train = MetaTask(train_examples, num_task = 100, k_support=100, k_query=30, tokenizer = tokenizer)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
#Take a glance at the first two samples from support set of 1st meta-task
train.supports[0][:2]

[{'text': 'I have been using the Family Tree Maker products for a number of years. I love its ease in the ability to add new people into the genealogy charts. I am a little disappointed in the change that moved the screen away from looking like a tabbed folder. I feel that the old style was a lot more user friendly, both visually and functionally',
  'label': 'positive',
  'domain': 'software'},
 {'text': "I checked around before making this purchase, and Amazon had the best price for Print Shop. I love it, I've used it for a couple of projects and I have no complaints. It's like a one stop shopping for all your publishing needs",
  'label': 'positive',
  'domain': 'software'}]

In [8]:
# Information of the 1st meta-task. It contains two TensorDataset: support set and query set
train[0]

(<torch.utils.data.dataset.TensorDataset at 0x7fce6e4d54d0>,
 <torch.utils.data.dataset.TensorDataset at 0x7fcdfe074b10>)

In [9]:
# Let take a look at the first two samples from support set
train[0][0][:2]

(tensor([[  101,  1045,  2031,  2042,  2478,  1996,  2155,  3392,  9338,  3688,
           2005,  1037,  2193,  1997,  2086,  1012,  1045,  2293,  2049,  7496,
           1999,  1996,  3754,  2000,  5587,  2047,  2111,  2046,  1996, 26684,
           6093,  1012,  1045,  2572,  1037,  2210,  9364,  1999,  1996,  2689,
           2008,  2333,  1996,  3898,  2185,  2013,  2559,  2066,  1037, 21628,
           8270, 19622,  1012,  1045,  2514,  2008,  1996,  2214,  2806,  2001,
           1037,  2843,  2062,  5310,  5379,  1010,  2119, 17453,  1998,  8360,
           2135,   102,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

###Training meta

In [10]:
import time
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

def random_seed(value):
    torch.backends.cudnn.deterministic=True
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    np.random.seed(value)
    random.seed(value)

def create_batch_of_tasks(taskset, is_shuffle = True, batch_size = 4):
    idxs = list(range(0,len(taskset)))
    if is_shuffle:
        random.shuffle(idxs)
    for i in range(0,len(idxs), batch_size):
        yield [taskset[idxs[i]] for i in range(i, min(i + batch_size,len(taskset)))]

class TrainingArgs:
    def __init__(self):
        self.num_labels = 2
        self.meta_epoch=10
        self.k_spt=80
        self.k_qry=20
        self.outer_batch_size = 2
        self.inner_batch_size = 12
        self.outer_update_lr = 5e-5
        self.inner_update_lr = 5e-5
        self.inner_update_step = 10
        self.inner_update_step_eval = 40
        self.bert_model = 'bert-base-uncased'
        self.num_task_train = 500
        self.num_task_test = 5

args = TrainingArgs()

###Create Meta Learner

In [11]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from transformers import BertForSequenceClassification
from copy import deepcopy
import gc
from sklearn.metrics import accuracy_score
import torch
import numpy as np

class Learner(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, args):
        """
        :param args:
        """
        super(Learner, self).__init__()
        
        self.num_labels = args.num_labels
        self.outer_batch_size = args.outer_batch_size
        self.inner_batch_size = args.inner_batch_size
        self.outer_update_lr  = args.outer_update_lr
        self.inner_update_lr  = args.inner_update_lr
        self.inner_update_step = args.inner_update_step
        self.inner_update_step_eval = args.inner_update_step_eval
        self.bert_model = args.bert_model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.model = BertForSequenceClassification.from_pretrained(self.bert_model, num_labels = self.num_labels)
        self.outer_optimizer = Adam(self.model.parameters(), lr=self.outer_update_lr)
        self.model.train()

    def forward(self, batch_tasks, training = True):
        """
        batch = [(support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset)]
        
        # support = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)
        """
        task_accs = []
        sum_gradients = []
        num_task = len(batch_tasks)
        num_inner_update_step = self.inner_update_step if training else self.inner_update_step_eval

        for task_id, task in enumerate(batch_tasks):
            support = task[0]
            query   = task[1]
            
            fast_model = deepcopy(self.model)
            fast_model.to(self.device)
            support_dataloader = DataLoader(support, sampler=RandomSampler(support),
                                            batch_size=self.inner_batch_size)
            
            inner_optimizer = Adam(fast_model.parameters(), lr=self.inner_update_lr)
            fast_model.train()
            
            print('----Task',task_id, '----')
            for i in range(0,num_inner_update_step):
                all_loss = []
                for inner_step, batch in enumerate(support_dataloader):
                    
                    batch = tuple(t.to(self.device) for t in batch)
                    input_ids, attention_mask, segment_ids, label_id = batch
                    outputs = fast_model(input_ids, attention_mask, segment_ids, labels = label_id)
                    
                    loss = outputs[0]              
                    loss.backward()
                    inner_optimizer.step()
                    inner_optimizer.zero_grad()
                    
                    all_loss.append(loss.item())
                
                if i % 4 == 0:
                    print("Inner Loss: ", np.mean(all_loss))
            
            fast_model.to(torch.device('cpu'))
            
            if training:
                meta_weights = list(self.model.parameters())
                fast_weights = list(fast_model.parameters())

                gradients = []
                for i, (meta_params, fast_params) in enumerate(zip(meta_weights, fast_weights)):
                    gradient = meta_params - fast_params
                    if task_id == 0:
                        sum_gradients.append(gradient)
                    else:
                        sum_gradients[i] += gradient

            fast_model.to(self.device)
            fast_model.eval()
            with torch.no_grad():
                query_dataloader = DataLoader(query, sampler=None, batch_size=len(query))
                query_batch = iter(query_dataloader).next()
                query_batch = tuple(t.to(self.device) for t in query_batch)
                q_input_ids, q_attention_mask, q_segment_ids, q_label_id = query_batch
                q_outputs = fast_model(q_input_ids, q_attention_mask, q_segment_ids, labels = q_label_id)

                q_logits = F.softmax(q_outputs[1],dim=1)
                pre_label_id = torch.argmax(q_logits,dim=1)
                pre_label_id = pre_label_id.detach().cpu().numpy().tolist()
                q_label_id = q_label_id.detach().cpu().numpy().tolist()

                acc = accuracy_score(pre_label_id,q_label_id)
                task_accs.append(acc)
            
            fast_model.to(torch.device('cpu'))
            del fast_model, inner_optimizer
            torch.cuda.empty_cache()
        
        if training:
            # Average gradient across tasks
            for i in range(0,len(sum_gradients)):
                sum_gradients[i] = sum_gradients[i] / float(num_task)

            #Assign gradient for original model, then using optimizer to update its weights
            for i, params in enumerate(self.model.parameters()):
                params.grad = sum_gradients[i]

            self.outer_optimizer.step()
            self.outer_optimizer.zero_grad()
            
            del sum_gradients
            gc.collect()
        
        return np.mean(task_accs)

In [12]:
learner = Learner(args)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
random_seed(123)
test = MetaTask(test_examples, num_task = 3, k_support=40, k_query=20, tokenizer = tokenizer)
random_seed(int(time.time() % 10))

In [14]:
test.supports[2]

[{'text': 'i have been playing the madden series since i was a kid and this game is really good. the games just keep getting better and better as each year passes. everything in this game is worth the money i strongly reccommend it',
  'label': 'positive',
  'domain': 'computer_&_video_games'},
 {'text': 'I noticed there were not many new items in this game, I found this the advertising for this game to be deceiving. This expansion ws not worth the money',
  'label': 'negative',
  'domain': 'computer_&_video_games'},
 {'text': "This pack offered very little of anything. The decor (very little of it) is uninspired. I was hoping to get some good artwork at least. I know a lot of people don't like to download from fan sites but don't waste your money on this junk. Believe me fansites have much better and more creative stuff for your sims and most of it is free, and none of it cost as much as this glamour life stuff pack did. This is the first and last stuff pack that I will ever buy",
  '

###Model Training

In [15]:
global_step = 0

for epoch in range(args.meta_epoch):
    
    train = MetaTask(train_examples, num_task = 500, k_support=80, k_query=20, tokenizer = tokenizer)
    db = create_batch_of_tasks(train, is_shuffle = True, batch_size = args.outer_batch_size)

    for step, task_batch in enumerate(db):
        
        f = open('log.txt', 'a')
        
        acc = learner(task_batch)
        
        print('Step:', step, '\ttraining Acc:', acc)
        f.write(str(acc) + '\n')
        
        if global_step % 20 == 0:
            random_seed(123)
            print("\n-----------------Testing Mode-----------------\n")
            db_test = create_batch_of_tasks(test, is_shuffle = False, batch_size = 1)
            acc_all_test = []

            for test_batch in db_test:
                acc = learner(test_batch, training = False)
                acc_all_test.append(acc)

            print('Step:', step, 'Test F1:', np.mean(acc_all_test))
            f.write('Test' + str(np.mean(acc_all_test)) + '\n')
            
            random_seed(int(time.time() % 10))
        
        global_step += 1
        f.close()

----Task 0 ----
Inner Loss:  0.6417855407510485
Inner Loss:  0.08291255310177803
Inner Loss:  0.01197606890595385
----Task 1 ----
Inner Loss:  0.6599393699850354
Inner Loss:  0.042447470660720556
Inner Loss:  0.007878676389477082
Step: 0 	training Acc: 0.85

-----------------Testing Mode-----------------

----Task 0 ----
Inner Loss:  0.6570065766572952
Inner Loss:  0.08210394810885191
Inner Loss:  0.02109398366883397
Inner Loss:  0.007071996689774096
Inner Loss:  0.003629075363278389
Inner Loss:  0.0022031054249964654
Inner Loss:  0.0012895008258055896
Inner Loss:  0.0011130327184218913
Inner Loss:  0.0008266328659374267
Inner Loss:  0.0007402164046652615
----Task 0 ----
Inner Loss:  0.6404849514365196
Inner Loss:  0.10750341229140759
Inner Loss:  0.028530654963105917
Inner Loss:  0.010565129341557622
Inner Loss:  0.006997286807745695
Inner Loss:  0.004740018863230944
Inner Loss:  0.003528118599206209
Inner Loss:  0.0028892927221022546
Inner Loss:  0.0025461220066063106
Inner Loss:  0.

KeyboardInterrupt: ignored

#Reference
###https://github.com/mailong25/meta-learning-bert